<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === GQR TDSE — Man2-LOCKED baseline + B variants (HC vs HCB) ===
# One cell; Colab/Jupyter ready; NumPy by default, CuPy if available.
# Outputs: PNG frames + MP4 for each condition (HC, HCB), plus a side-by-side MP4.

# -------------------- Setup --------------------
!pip -q install imageio-ffmpeg

import os, math, csv, json, sys
from dataclasses import dataclass

# ---- Backend select (GPU if available); physics computed identically ----
USE_GPU = False
try:
    import cupy as cp
    _ = cp.zeros((1,))
    USE_GPU = True
except Exception:
    cp = None

import numpy as np
xp = cp if USE_GPU else np

import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")  # headless-safe

import imageio.v2 as iio

print(f"[backend] {'CuPy (GPU)' if USE_GPU else 'NumPy (CPU)'}")

# -------------------- Helpers --------------------
def ensure_dir(p): os.makedirs(p, exist_ok=True)

def to_cpu(a):
    """Return as NumPy array, regardless of backend."""
    if USE_GPU and isinstance(a, cp.ndarray):
        return cp.asnumpy(a)
    return np.asarray(a)

def pad_to_macroblocks(rgb, block=16):
    """Pad HxWx3 uint8 array so H and W are multiples of `block`."""
    h, w, c = rgb.shape
    H = ((h + block - 1)//block)*block
    W = ((w + block - 1)//block)*block
    if H==h and W==w: return rgb
    pad = np.zeros((H, W, c), dtype=rgb.dtype)
    pad[:h,:w,:] = rgb
    return pad

def save_mp4_from_pngs(png_list, mp4_path, fps=12):
    with iio.get_writer(mp4_path, fps=fps, codec="libx264") as w:
        for p in png_list:
            frame = iio.imread(p)
            if frame.dtype != np.uint8:
                frame = np.clip(frame, 0, 255).astype(np.uint8)
            frame = pad_to_macroblocks(frame, 16)
            w.append_data(frame)

# -------------------- Man2-LOCKED parameters (DO NOT CHANGE) --------------------
# These match the earlier Man2 runs: same barrier, funnel, field, dt, grid, absorber.
MAN2_LOCK = dict(
    # Grid / time
    Nx=256, Ny=192, Lx=20.0, Ly=20.0,
    dt=0.01, T_final=60.0,          # long run (increase T_final for more frames)
    save_every=10,                  # save every 0.1 time unit (T_final/dt/save_every frames)
    mass=1.0, hbar=1.0,

    # Absorber (to prevent box reflections)
    absorb_frac=0.10, absorb_power=4,

    # Initial packet (left -> right)
    x0=-6.0, y0=0.0, sigma=1.1, kx=2.0, ky=0.0,

    # Baseline barrier A (Gaussian along x + gentle y confine)
    A_height=8.0, A_width=1.2, Vy_coeff=0.02,

    # Funnel H (aperture) — Gaussian notch to the +x side
    H_depth=4.0, H_x0=+3.0, H_y_sigma=1.2,

    # Field C (lens) — linear potential along +x
    C_strength=0.60,

    # Tilt B (prism) — linear bias along y (WE ONLY CHANGE B FOR VARIANT)
    B_strength=0.30,  # will be used in HCB only; HC uses B=0.0

    # Dephasing E OFF for Man3 figures focusing on B (buzz comes from H+C)
    E_gamma=0.0
)

# -------------------- Physics --------------------
@dataclass
class Grid:
    Nx: int; Ny: int; Lx: float; Ly: float; hbar: float; mass: float
    # built fields
    x: xp.ndarray=None; y: xp.ndarray=None
    X: xp.ndarray=None; Y: xp.ndarray=None
    kx: xp.ndarray=None; ky: xp.ndarray=None
    K2: xp.ndarray=None; ABSORB: xp.ndarray=None

def build_grid(cfg):
    Nx, Ny, Lx, Ly = cfg["Nx"], cfg["Ny"], cfg["Lx"], cfg["Ly"]
    x = xp.linspace(-Lx/2, Lx/2, Nx, endpoint=False)
    y = xp.linspace(-Ly/2, Ly/2, Ny, endpoint=False)
    X, Y = xp.meshgrid(x, y, indexing='xy')
    dx = float(x[1]-x[0]); dy = float(y[1]-y[0])
    kx = 2*xp.pi*(xp.fft.fftfreq(Nx, d=dx))
    ky = 2*xp.pi*(xp.fft.fftfreq(Ny, d=dy))
    KX, KY = xp.meshgrid(kx, ky, indexing='xy')
    K2 = KX**2 + KY**2

    # absorber
    afx = max(int(cfg["absorb_frac"]*Nx), 1)
    afy = max(int(cfg["absorb_frac"]*Ny), 1)
    wx = xp.ones(Nx); wy = xp.ones(Ny)
    ramp_x = xp.linspace(0, 1, afx)
    wx[:afx]  = xp.cos(0.5*xp.pi*(1 - ramp_x))**cfg["absorb_power"]
    wx[-afx:] = xp.cos(0.5*xp.pi*(ramp_x))**cfg["absorb_power"]
    ramp_y = xp.linspace(0, 1, afy)
    wy[:afy]  = xp.cos(0.5*xp.pi*(1 - ramp_y))**cfg["absorb_power"]
    wy[-afy:] = xp.cos(0.5*xp.pi*(ramp_y))**cfg["absorb_power"]
    ABSORB = xp.outer(wy, wx)  # (Ny,Nx)

    return Grid(Nx, Ny, Lx, Ly, cfg["hbar"], cfg["mass"], x, y, X, Y, kx, ky, K2, ABSORB)

def potential_A(G, cfg):
    Vx = cfg["A_height"] * xp.exp(-(G.X/cfg["A_width"])**2)
    Vy = cfg["Vy_coeff"] * (G.Y**2)
    return Vx + Vy

def gate_H(G, cfg):
    # localized Gaussian notch (aperture) on +x side
    return -cfg["H_depth"] * xp.exp(-((G.X - cfg["H_x0"])**2 + (G.Y/cfg["H_y_sigma"])**2))

def gate_C(G, cfg):
    return -cfg["C_strength"] * G.X

def gate_B(G, cfg, B_strength):
    return B_strength * (G.Y / (cfg["Ly"]/2.0))  # linear tilt along +y

def gaussian_packet(G, cfg):
    gauss = xp.exp(-((G.X - cfg["x0"])**2 + (G.Y - cfg["y0"])**2)/(2*cfg["sigma"]**2))
    phase = xp.exp(1j*(cfg["kx"]*G.X + cfg["ky"]*G.Y)/cfg["hbar"])
    psi = gauss * phase
    psi /= xp.sqrt(xp.sum(xp.abs(psi)**2))
    return psi

def precompute_T_phase(G, cfg):
    dt = cfg["dt"]; hbar = cfg["hbar"]; m = cfg["mass"]
    return xp.exp(-1j * (hbar**2 * G.K2) * dt / (2.0*m*hbar))

def split_op_step(psi, V, Tph, cfg):
    # half kinetic
    psi_k = xp.fft.fft2(psi)
    psi_k *= Tph
    psi = xp.fft.ifft2(psi_k)
    # potential
    psi *= xp.exp(-1j * V * cfg["dt"])
    # half kinetic
    psi_k = xp.fft.fft2(psi)
    psi_k *= Tph
    psi = xp.fft.ifft2(psi_k)
    return psi

# -------------------- Rendering --------------------
def render_density_png(dens, G, tag, i, out_dir, vmax=None, dpi=140):
    """Save |psi|^2 as PNG, padded to macroblocks for clean video."""
    extent = [float(to_cpu(G.x).min()), float(to_cpu(G.x).max()),
              float(to_cpu(G.y).min()), float(to_cpu(G.y).max())]
    d = to_cpu(dens)
    if vmax is None:
        vmax = max(np.percentile(d, 99.5), 1e-8)
    fig, ax = plt.subplots(figsize=(5.0, 3.6), dpi=dpi)  # ~700x504 px
    im = ax.imshow(d.T, origin='lower', extent=extent, vmin=0, vmax=vmax)
    ax.set_title(f"{tag}  |psi|^2")
    ax.set_xlabel("x"); ax.set_ylabel("y")
    cbar = fig.colorbar(im, ax=ax, shrink=0.8)
    # Draw and extract RGB
    fig.canvas.draw()
    w, h = fig.canvas.get_width_height()
    rgb = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8).reshape(h, w, 4)[:, :, :3]
    plt.close(fig)
    rgb = pad_to_macroblocks(rgb, 16)
    fn = os.path.join(out_dir, f"{tag}_psi2_{i:04d}.png")
    iio.imwrite(fn, rgb)
    return fn

# -------------------- Condition builders --------------------
def make_V_HC(G, cfg):
    V = potential_A(G, cfg) + gate_H(G, cfg) + gate_C(G, cfg)
    return V

def make_V_HCB(G, cfg):
    V = potential_A(G, cfg) + gate_H(G, cfg) + gate_C(G, cfg) + gate_B(G, cfg, cfg["B_strength"])
    return V

# -------------------- Runner --------------------
def run_condition(tag, V, G, cfg, out_dir):
    ensure_dir(out_dir)
    psi = gaussian_packet(G, cfg)
    Tph = precompute_T_phase(G, cfg)
    frames_png = []
    nsteps = int(cfg["T_final"]/cfg["dt"])
    vmax = None
    for n in range(nsteps):
        # Symmetric split-operator with absorber
        psi = split_op_step(psi, V, Tph, cfg)
        psi *= G.ABSORB
        if (n % cfg["save_every"]) == 0 or n == nsteps-1:
            dens = xp.abs(psi)**2
            png = render_density_png(dens, G, tag, n//cfg["save_every"], out_dir, vmax=vmax)
            frames_png.append(png)
            if vmax is None:
                vmax = max(np.percentile(to_cpu(dens), 99.5), 1e-8)  # lock after first frame
    return frames_png

# -------------------- Main --------------------
OUT_ROOT = "man2locked_HC_vs_HCB"
ensure_dir(OUT_ROOT)

# Freeze config (exact Man2)
cfg = MAN2_LOCK.copy()

# Build grid/potentials once
G = build_grid(cfg)
V_HC  = make_V_HC(G, cfg)
V_HCB = make_V_HCB(G, cfg)

# Run HC
print("[run] HC (Man2-locked)")
HC_DIR = os.path.join(OUT_ROOT, "HC")
pngs_HC = run_condition("HC", V_HC, G, cfg, HC_DIR)
mp4_HC = os.path.join(HC_DIR, "HC.mp4")
save_mp4_from_pngs(pngs_HC, mp4_HC, fps=12)
print(f"[ok] HC frames: {len(pngs_HC)}  MP4: {mp4_HC}")

# Run HCB
print("[run] HCB (Man2-locked + B)")
HCB_DIR = os.path.join(OUT_ROOT, "HCB")
pngs_HCB = run_condition("HCB", V_HCB, G, cfg, HCB_DIR)
mp4_HCB = os.path.join(HCB_DIR, "HCB.mp4")
save_mp4_from_pngs(pngs_HCB, mp4_HCB, fps=12)
print(f"[ok] HCB frames: {len(pngs_HCB)}  MP4: {mp4_HCB}")

# Side-by-side comparison MP4 (same frame index; assumes equal counts)
print("[make] side-by-side HC|HCB")
SBS_DIR = os.path.join(OUT_ROOT, "side_by_side"); ensure_dir(SBS_DIR)
sbs_mp4 = os.path.join(SBS_DIR, "HC_vs_HCB.mp4")
pairs = zip(pngs_HC, pngs_HCB)
with iio.get_writer(sbs_mp4, fps=12, codec="libx264") as w:
    for a,b in pairs:
        A = iio.imread(a); B = iio.imread(b)
        # equalize heights, then hstack
        h = max(A.shape[0], B.shape[0])
        wA, wB = A.shape[1], B.shape[1]
        padA = np.zeros((h, wA, 3), dtype=np.uint8); padA[:A.shape[0], :wA] = A
        padB = np.zeros((h, wB, 3), dtype=np.uint8); padB[:B.shape[0], :wB] = B
        sbs = np.hstack([padA, padB])
        sbs = pad_to_macroblocks(sbs, 16)
        w.append_data(sbs)
print(f"[ok] Side-by-side: {sbs_mp4}")

print("\n=== DONE ===")
print("Output root:", OUT_ROOT)
print("HC MP4     :", mp4_HC)
print("HCB MP4    :", mp4_HCB)
print("SBS MP4    :", sbs_mp4)

[backend] CuPy (GPU)
[run] HC (Man2-locked)
[ok] HC frames: 601  MP4: man2locked_HC_vs_HCB/HC/HC.mp4
[run] HCB (Man2-locked + B)
[ok] HCB frames: 601  MP4: man2locked_HC_vs_HCB/HCB/HCB.mp4
[make] side-by-side HC|HCB
[ok] Side-by-side: man2locked_HC_vs_HCB/side_by_side/HC_vs_HCB.mp4

=== DONE ===
Output root: man2locked_HC_vs_HCB
HC MP4     : man2locked_HC_vs_HCB/HC/HC.mp4
HCB MP4    : man2locked_HC_vs_HCB/HCB/HCB.mp4
SBS MP4    : man2locked_HC_vs_HCB/side_by_side/HC_vs_HCB.mp4


In [ ]:
# === GQR TDSE — Man2-LOCKED baseline + B variants (HC vs HCB) ===
# One cell; Colab/Jupyter ready; NumPy by default, CuPy if available.
# Outputs: PNG frames + MP4 for each condition (HC, HCB), plus a side-by-side MP4.

# -------------------- Setup --------------------
!pip -q install imageio-ffmpeg

import os, math, csv, json, sys
from dataclasses import dataclass

# ---- Backend select (GPU if available); physics computed identically ----
USE_GPU = False
try:
    import cupy as cp
    _ = cp.zeros((1,))
    USE_GPU = True
except Exception:
    cp = None

import numpy as np
xp = cp if USE_GPU else np

import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")  # headless-safe

import imageio.v2 as iio

print(f"[backend] {'CuPy (GPU)' if USE_GPU else 'NumPy (CPU)'}")

# -------------------- Helpers --------------------
def ensure_dir(p): os.makedirs(p, exist_ok=True)

def to_cpu(a):
    """Return as NumPy array, regardless of backend."""
    if USE_GPU and isinstance(a, cp.ndarray):
        return cp.asnumpy(a)
    return np.asarray(a)

def pad_to_macroblocks(rgb, block=16):
    """Pad HxWx3 uint8 array so H and W are multiples of `block`."""
    h, w, c = rgb.shape
    H = ((h + block - 1)//block)*block
    W = ((w + block - 1)//block)*block
    if H==h and W==w: return rgb
    pad = np.zeros((H, W, c), dtype=rgb.dtype)
    pad[:h,:w,:] = rgb
    return pad

def save_mp4_from_pngs(png_list, mp4_path, fps=12):
    with iio.get_writer(mp4_path, fps=fps, codec="libx264") as w:
        for p in png_list:
            frame = iio.imread(p)
            if frame.dtype != np.uint8:
                frame = np.clip(frame, 0, 255).astype(np.uint8)
            frame = pad_to_macroblocks(frame, 16)
            w.append_data(frame)

# -------------------- Man2-LOCKED parameters (DO NOT CHANGE) --------------------
# These match the earlier Man2 runs: same barrier, funnel, field, dt, grid, absorber.
MAN2_LOCK = dict(
    # Grid / time
    Nx=256, Ny=192, Lx=20.0, Ly=20.0,
    dt=0.01, T_final=60.0,          # long run (increase T_final for more frames)
    save_every=10,                  # save every 0.1 time unit (T_final/dt/save_every frames)
    mass=1.0, hbar=1.0,

    # Absorber (to prevent box reflections)
    absorb_frac=0.10, absorb_power=4,

    # Initial packet (left -> right)
    x0=-6.0, y0=0.0, sigma=1.1, kx=2.0, ky=0.0,

    # Baseline barrier A (Gaussian along x + gentle y confine)
    A_height=8.0, A_width=1.2, Vy_coeff=0.02,

    # Funnel H (aperture) — Gaussian notch to the +x side
    H_depth=4.0, H_x0=+3.0, H_y_sigma=1.2,

    # Field C (lens) — linear potential along +x
    C_strength=0.60,

    # Tilt B (prism) — linear bias along y (WE ONLY CHANGE B FOR VARIANT)
    B_strength=0.30,  # will be used in HCB only; HC uses B=0.0

    # Dephasing E OFF for Man3 figures focusing on B (buzz comes from H+C)
    E_gamma=0.0
)

# -------------------- Physics --------------------
@dataclass
class Grid:
    Nx: int; Ny: int; Lx: float; Ly: float; hbar: float; mass: float
    # built fields
    x: xp.ndarray=None; y: xp.ndarray=None
    X: xp.ndarray=None; Y: xp.ndarray=None
    kx: xp.ndarray=None; ky: xp.ndarray=None
    K2: xp.ndarray=None; ABSORB: xp.ndarray=None

def build_grid(cfg):
    Nx, Ny, Lx, Ly = cfg["Nx"], cfg["Ny"], cfg["Lx"], cfg["Ly"]
    x = xp.linspace(-Lx/2, Lx/2, Nx, endpoint=False)
    y = xp.linspace(-Ly/2, Ly/2, Ny, endpoint=False)
    X, Y = xp.meshgrid(x, y, indexing='xy')
    dx = float(x[1]-x[0]); dy = float(y[1]-y[0])
    kx = 2*xp.pi*(xp.fft.fftfreq(Nx, d=dx))
    ky = 2*xp.pi*(xp.fft.fftfreq(Ny, d=dy))
    KX, KY = xp.meshgrid(kx, ky, indexing='xy')
    K2 = KX**2 + KY**2

    # absorber
    afx = max(int(cfg["absorb_frac"]*Nx), 1)
    afy = max(int(cfg["absorb_frac"]*Ny), 1)
    wx = xp.ones(Nx); wy = xp.ones(Ny)
    ramp_x = xp.linspace(0, 1, afx)
    wx[:afx]  = xp.cos(0.5*xp.pi*(1 - ramp_x))**cfg["absorb_power"]
    wx[-afx:] = xp.cos(0.5*xp.pi*(ramp_x))**cfg["absorb_power"]
    ramp_y = xp.linspace(0, 1, afy)
    wy[:afy]  = xp.cos(0.5*xp.pi*(1 - ramp_y))**cfg["absorb_power"]
    wy[-afy:] = xp.cos(0.5*xp.pi*(ramp_y))**cfg["absorb_power"]
    ABSORB = xp.outer(wy, wx)  # (Ny,Nx)

    return Grid(Nx, Ny, Lx, Ly, cfg["hbar"], cfg["mass"], x, y, X, Y, kx, ky, K2, ABSORB)

def potential_A(G, cfg):
    Vx = cfg["A_height"] * xp.exp(-(G.X/cfg["A_width"])**2)
    Vy = cfg["Vy_coeff"] * (G.Y**2)
    return Vx + Vy

def gate_H(G, cfg):
    # localized Gaussian notch (aperture) on +x side
    return -cfg["H_depth"] * xp.exp(-((G.X - cfg["H_x0"])**2 + (G.Y/cfg["H_y_sigma"])**2))

def gate_C(G, cfg):
    return -cfg["C_strength"] * G.X

def gate_B(G, cfg, B_strength):
    return B_strength * (G.Y / (cfg["Ly"]/2.0))  # linear tilt along +y

def gaussian_packet(G, cfg):
    gauss = xp.exp(-((G.X - cfg["x0"])**2 + (G.Y - cfg["y0"])**2)/(2*cfg["sigma"]**2))
    phase = xp.exp(1j*(cfg["kx"]*G.X + cfg["ky"]*G.Y)/cfg["hbar"])
    psi = gauss * phase
    psi /= xp.sqrt(xp.sum(xp.abs(psi)**2))
    return psi

def precompute_T_phase(G, cfg):
    dt = cfg["dt"]; hbar = cfg["hbar"]; m = cfg["mass"]
    return xp.exp(-1j * (hbar**2 * G.K2) * dt / (2.0*m*hbar))

def split_op_step(psi, V, Tph, cfg):
    # half kinetic
    psi_k = xp.fft.fft2(psi)
    psi_k *= Tph
    psi = xp.fft.ifft2(psi_k)
    # potential
    psi *= xp.exp(-1j * V * cfg["dt"])
    # half kinetic
    psi_k = xp.fft.fft2(psi)
    psi_k *= Tph
    psi = xp.fft.ifft2(psi_k)
    return psi

# -------------------- Rendering --------------------
def render_density_png(dens, G, tag, i, out_dir, vmax=None, dpi=140):
    """Save |psi|^2 as PNG, padded to macroblocks for clean video."""
    extent = [float(to_cpu(G.x).min()), float(to_cpu(G.x).max()),
              float(to_cpu(G.y).min()), float(to_cpu(G.y).max())]
    d = to_cpu(dens)
    if vmax is None:
        vmax = max(np.percentile(d, 99.5), 1e-8)
    fig, ax = plt.subplots(figsize=(5.0, 3.6), dpi=dpi)  # ~700x504 px
    im = ax.imshow(d.T, origin='lower', extent=extent, vmin=0, vmax=vmax)
    ax.set_title(f"{tag}  |psi|^2")
    ax.set_xlabel("x"); ax.set_ylabel("y")
    cbar = fig.colorbar(im, ax=ax, shrink=0.8)
    # Draw and extract RGB
    fig.canvas.draw()
    w, h = fig.canvas.get_width_height()
    rgb = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8).reshape(h, w, 4)[:, :, :3]
    plt.close(fig)
    rgb = pad_to_macroblocks(rgb, 16)
    fn = os.path.join(out_dir, f"{tag}_psi2_{i:04d}.png")
    iio.imwrite(fn, rgb)
    return fn

# -------------------- Condition builders --------------------
def make_V_HC(G, cfg):
    V = potential_A(G, cfg) + gate_H(G, cfg) + gate_C(G, cfg)
    return V

def make_V_HCB(G, cfg):
    V = potential_A(G, cfg) + gate_H(G, cfg) + gate_C(G, cfg) + gate_B(G, cfg, cfg["B_strength"])
    return V

# -------------------- Runner --------------------
def run_condition(tag, V, G, cfg, out_dir):
    ensure_dir(out_dir)
    psi = gaussian_packet(G, cfg)
    Tph = precompute_T_phase(G, cfg)
    frames_png = []
    nsteps = int(cfg["T_final"]/cfg["dt"])
    vmax = None
    for n in range(nsteps):
        # Symmetric split-operator with absorber
        psi = split_op_step(psi, V, Tph, cfg)
        psi *= G.ABSORB
        if (n % cfg["save_every"]) == 0 or n == nsteps-1:
            dens = xp.abs(psi)**2
            png = render_density_png(dens, G, tag, n//cfg["save_every"], out_dir, vmax=vmax)
            frames_png.append(png)
            if vmax is None:
                vmax = max(np.percentile(to_cpu(dens), 99.5), 1e-8)  # lock after first frame
    return frames_png

# -------------------- Main --------------------
OUT_ROOT = "man2locked_HC_vs_HCB"
ensure_dir(OUT_ROOT)

# Freeze config (exact Man2)
cfg = MAN2_LOCK.copy()

# Build grid/potentials once
G = build_grid(cfg)
V_HC  = make_V_HC(G, cfg)
V_HCB = make_V_HCB(G, cfg)

# Run HC
print("[run] HC (Man2-locked)")
HC_DIR = os.path.join(OUT_ROOT, "HC")
pngs_HC = run_condition("HC", V_HC, G, cfg, HC_DIR)
mp4_HC = os.path.join(HC_DIR, "HC.mp4")
save_mp4_from_pngs(pngs_HC, mp4_HC, fps=12)
print(f"[ok] HC frames: {len(pngs_HC)}  MP4: {mp4_HC}")

# Run HCB
print("[run] HCB (Man2-locked + B)")
HCB_DIR = os.path.join(OUT_ROOT, "HCB")
pngs_HCB = run_condition("HCB", V_HCB, G, cfg, HCB_DIR)
mp4_HCB = os.path.join(HCB_DIR, "HCB.mp4")
save_mp4_from_pngs(pngs_HCB, mp4_HCB, fps=12)
print(f"[ok] HCB frames: {len(pngs_HCB)}  MP4: {mp4_HCB}")

# Side-by-side comparison MP4 (same frame index; assumes equal counts)
print("[make] side-by-side HC|HCB")
SBS_DIR = os.path.join(OUT_ROOT, "side_by_side"); ensure_dir(SBS_DIR)
sbs_mp4 = os.path.join(SBS_DIR, "HC_vs_HCB.mp4")
pairs = zip(pngs_HC, pngs_HCB)
with iio.get_writer(sbs_mp4, fps=12, codec="libx264") as w:
    for a,b in pairs:
        A = iio.imread(a); B = iio.imread(b)
        # equalize heights, then hstack
        h = max(A.shape[0], B.shape[0])
        wA, wB = A.shape[1], B.shape[1]
        padA = np.zeros((h, wA, 3), dtype=np.uint8); padA[:A.shape[0], :wA] = A
        padB = np.zeros((h, wB, 3), dtype=np.uint8); padB[:B.shape[0], :wB] = B
        sbs = np.hstack([padA, padB])
        sbs = pad_to_macroblocks(sbs, 16)
        w.append_data(sbs)
print(f"[ok] Side-by-side: {sbs_mp4}")

print("\n=== DONE ===")
print("Output root:", OUT_ROOT)
print("HC MP4     :", mp4_HC)
print("HCB MP4    :", mp4_HCB)
print("SBS MP4    :", sbs_mp4)

[backend] CuPy (GPU)
[run] HC (Man2-locked)
[ok] HC frames: 601  MP4: man2locked_HC_vs_HCB/HC/HC.mp4
[run] HCB (Man2-locked + B)
[ok] HCB frames: 601  MP4: man2locked_HC_vs_HCB/HCB/HCB.mp4
[make] side-by-side HC|HCB
[ok] Side-by-side: man2locked_HC_vs_HCB/side_by_side/HC_vs_HCB.mp4

=== DONE ===
Output root: man2locked_HC_vs_HCB
HC MP4     : man2locked_HC_vs_HCB/HC/HC.mp4
HCB MP4    : man2locked_HC_vs_HCB/HCB/HCB.mp4
SBS MP4    : man2locked_HC_vs_HCB/side_by_side/HC_vs_HCB.mp4


In [ ]:
# === GQR TDSE — Man2-LOCKED baseline + B variants (HC vs HCB) ===
# One cell; Colab/Jupyter ready; NumPy by default, CuPy if available.
# Outputs: PNG frames + MP4 for each condition (HC, HCB), plus a side-by-side MP4.

# -------------------- Setup --------------------
!pip -q install imageio-ffmpeg

import os, math, csv, json, sys
from dataclasses import dataclass

# ---- Backend select (GPU if available); physics computed identically ----
USE_GPU = False
try:
    import cupy as cp
    _ = cp.zeros((1,))
    USE_GPU = True
except Exception:
    cp = None

import numpy as np
xp = cp if USE_GPU else np

import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")  # headless-safe

import imageio.v2 as iio

print(f"[backend] {'CuPy (GPU)' if USE_GPU else 'NumPy (CPU)'}")

# -------------------- Helpers --------------------
def ensure_dir(p): os.makedirs(p, exist_ok=True)

def to_cpu(a):
    """Return as NumPy array, regardless of backend."""
    if USE_GPU and isinstance(a, cp.ndarray):
        return cp.asnumpy(a)
    return np.asarray(a)

def pad_to_macroblocks(rgb, block=16):
    """Pad HxWx3 uint8 array so H and W are multiples of `block`."""
    h, w, c = rgb.shape
    H = ((h + block - 1)//block)*block
    W = ((w + block - 1)//block)*block
    if H==h and W==w: return rgb
    pad = np.zeros((H, W, c), dtype=rgb.dtype)
    pad[:h,:w,:] = rgb
    return pad

def save_mp4_from_pngs(png_list, mp4_path, fps=12):
    with iio.get_writer(mp4_path, fps=fps, codec="libx264") as w:
        for p in png_list:
            frame = iio.imread(p)
            if frame.dtype != np.uint8:
                frame = np.clip(frame, 0, 255).astype(np.uint8)
            frame = pad_to_macroblocks(frame, 16)
            w.append_data(frame)

# -------------------- Man2-LOCKED parameters (DO NOT CHANGE) --------------------
# These match the earlier Man2 runs: same barrier, funnel, field, dt, grid, absorber.
MAN2_LOCK = dict(
    # Grid / time
    Nx=256, Ny=192, Lx=20.0, Ly=20.0,
    dt=0.01, T_final=60.0,          # long run (increase T_final for more frames)
    save_every=10,                  # save every 0.1 time unit (T_final/dt/save_every frames)
    mass=1.0, hbar=1.0,

    # Absorber (to prevent box reflections)
    absorb_frac=0.10, absorb_power=4,

    # Initial packet (left -> right)
    x0=-6.0, y0=0.0, sigma=1.1, kx=2.0, ky=0.0,

    # Baseline barrier A (Gaussian along x + gentle y confine)
    A_height=8.0, A_width=1.2, Vy_coeff=0.02,

    # Funnel H (aperture) — Gaussian notch to the +x side
    H_depth=4.0, H_x0=+3.0, H_y_sigma=1.2,

    # Field C (lens) — linear potential along +x
    C_strength=0.60,

    # Tilt B (prism) — linear bias along y (WE ONLY CHANGE B FOR VARIANT)
    B_strength=0.30,  # will be used in HCB only; HC uses B=0.0

    # Dephasing E OFF for Man3 figures focusing on B (buzz comes from H+C)
    E_gamma=0.0
)

# -------------------- Physics --------------------
@dataclass
class Grid:
    Nx: int; Ny: int; Lx: float; Ly: float; hbar: float; mass: float
    # built fields
    x: xp.ndarray=None; y: xp.ndarray=None
    X: xp.ndarray=None; Y: xp.ndarray=None
    kx: xp.ndarray=None; ky: xp.ndarray=None
    K2: xp.ndarray=None; ABSORB: xp.ndarray=None

def build_grid(cfg):
    Nx, Ny, Lx, Ly = cfg["Nx"], cfg["Ny"], cfg["Lx"], cfg["Ly"]
    x = xp.linspace(-Lx/2, Lx/2, Nx, endpoint=False)
    y = xp.linspace(-Ly/2, Ly/2, Ny, endpoint=False)
    X, Y = xp.meshgrid(x, y, indexing='xy')
    dx = float(x[1]-x[0]); dy = float(y[1]-y[0])
    kx = 2*xp.pi*(xp.fft.fftfreq(Nx, d=dx))
    ky = 2*xp.pi*(xp.fft.fftfreq(Ny, d=dy))
    KX, KY = xp.meshgrid(kx, ky, indexing='xy')
    K2 = KX**2 + KY**2

    # absorber
    afx = max(int(cfg["absorb_frac"]*Nx), 1)
    afy = max(int(cfg["absorb_frac"]*Ny), 1)
    wx = xp.ones(Nx); wy = xp.ones(Ny)
    ramp_x = xp.linspace(0, 1, afx)
    wx[:afx]  = xp.cos(0.5*xp.pi*(1 - ramp_x))**cfg["absorb_power"]
    wx[-afx:] = xp.cos(0.5*xp.pi*(ramp_x))**cfg["absorb_power"]
    ramp_y = xp.linspace(0, 1, afy)
    wy[:afy]  = xp.cos(0.5*xp.pi*(1 - ramp_y))**cfg["absorb_power"]
    wy[-afy:] = xp.cos(0.5*xp.pi*(ramp_y))**cfg["absorb_power"]
    ABSORB = xp.outer(wy, wx)  # (Ny,Nx)

    return Grid(Nx, Ny, Lx, Ly, cfg["hbar"], cfg["mass"], x, y, X, Y, kx, ky, K2, ABSORB)

def potential_A(G, cfg):
    Vx = cfg["A_height"] * xp.exp(-(G.X/cfg["A_width"])**2)
    Vy = cfg["Vy_coeff"] * (G.Y**2)
    return Vx + Vy

def gate_H(G, cfg):
    # localized Gaussian notch (aperture) on +x side
    return -cfg["H_depth"] * xp.exp(-((G.X - cfg["H_x0"])**2 + (G.Y/cfg["H_y_sigma"])**2))

def gate_C(G, cfg):
    return -cfg["C_strength"] * G.X

def gate_B(G, cfg, B_strength):
    return B_strength * (G.Y / (cfg["Ly"]/2.0))  # linear tilt along +y

def gaussian_packet(G, cfg):
    gauss = xp.exp(-((G.X - cfg["x0"])**2 + (G.Y - cfg["y0"])**2)/(2*cfg["sigma"]**2))
    phase = xp.exp(1j*(cfg["kx"]*G.X + cfg["ky"]*G.Y)/cfg["hbar"])
    psi = gauss * phase
    psi /= xp.sqrt(xp.sum(xp.abs(psi)**2))
    return psi

def precompute_T_phase(G, cfg):
    dt = cfg["dt"]; hbar = cfg["hbar"]; m = cfg["mass"]
    return xp.exp(-1j * (hbar**2 * G.K2) * dt / (2.0*m*hbar))

def split_op_step(psi, V, Tph, cfg):
    # half kinetic
    psi_k = xp.fft.fft2(psi)
    psi_k *= Tph
    psi = xp.fft.ifft2(psi_k)
    # potential
    psi *= xp.exp(-1j * V * cfg["dt"])
    # half kinetic
    psi_k = xp.fft.fft2(psi)
    psi_k *= Tph
    psi = xp.fft.ifft2(psi_k)
    return psi

# -------------------- Rendering --------------------
def render_density_png(dens, G, tag, i, out_dir, vmax=None, dpi=140):
    """Save |psi|^2 as PNG, padded to macroblocks for clean video."""
    extent = [float(to_cpu(G.x).min()), float(to_cpu(G.x).max()),
              float(to_cpu(G.y).min()), float(to_cpu(G.y).max())]
    d = to_cpu(dens)
    if vmax is None:
        vmax = max(np.percentile(d, 99.5), 1e-8)
    fig, ax = plt.subplots(figsize=(5.0, 3.6), dpi=dpi)  # ~700x504 px
    im = ax.imshow(d.T, origin='lower', extent=extent, vmin=0, vmax=vmax)
    ax.set_title(f"{tag}  |psi|^2")
    ax.set_xlabel("x"); ax.set_ylabel("y")
    cbar = fig.colorbar(im, ax=ax, shrink=0.8)
    # Draw and extract RGB
    fig.canvas.draw()
    w, h = fig.canvas.get_width_height()
    rgb = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8).reshape(h, w, 4)[:, :, :3]
    plt.close(fig)
    rgb = pad_to_macroblocks(rgb, 16)
    fn = os.path.join(out_dir, f"{tag}_psi2_{i:04d}.png")
    iio.imwrite(fn, rgb)
    return fn

# -------------------- Condition builders --------------------
def make_V_HC(G, cfg):
    V = potential_A(G, cfg) + gate_H(G, cfg) + gate_C(G, cfg)
    return V

def make_V_HCB(G, cfg):
    V = potential_A(G, cfg) + gate_H(G, cfg) + gate_C(G, cfg) + gate_B(G, cfg, cfg["B_strength"])
    return V

# -------------------- Runner --------------------
def run_condition(tag, V, G, cfg, out_dir):
    ensure_dir(out_dir)
    psi = gaussian_packet(G, cfg)
    Tph = precompute_T_phase(G, cfg)
    frames_png = []
    nsteps = int(cfg["T_final"]/cfg["dt"])
    vmax = None
    for n in range(nsteps):
        # Symmetric split-operator with absorber
        psi = split_op_step(psi, V, Tph, cfg)
        psi *= G.ABSORB
        if (n % cfg["save_every"]) == 0 or n == nsteps-1:
            dens = xp.abs(psi)**2
            png = render_density_png(dens, G, tag, n//cfg["save_every"], out_dir, vmax=vmax)
            frames_png.append(png)
            if vmax is None:
                vmax = max(np.percentile(to_cpu(dens), 99.5), 1e-8)  # lock after first frame
    return frames_png

# -------------------- Main --------------------
OUT_ROOT = "man2locked_HC_vs_HCB"
ensure_dir(OUT_ROOT)

# Freeze config (exact Man2)
cfg = MAN2_LOCK.copy()

# Build grid/potentials once
G = build_grid(cfg)
V_HC  = make_V_HC(G, cfg)
V_HCB = make_V_HCB(G, cfg)

# Run HC
print("[run] HC (Man2-locked)")
HC_DIR = os.path.join(OUT_ROOT, "HC")
pngs_HC = run_condition("HC", V_HC, G, cfg, HC_DIR)
mp4_HC = os.path.join(HC_DIR, "HC.mp4")
save_mp4_from_pngs(pngs_HC, mp4_HC, fps=12)
print(f"[ok] HC frames: {len(pngs_HC)}  MP4: {mp4_HC}")

# Run HCB
print("[run] HCB (Man2-locked + B)")
HCB_DIR = os.path.join(OUT_ROOT, "HCB")
pngs_HCB = run_condition("HCB", V_HCB, G, cfg, HCB_DIR)
mp4_HCB = os.path.join(HCB_DIR, "HCB.mp4")
save_mp4_from_pngs(pngs_HCB, mp4_HCB, fps=12)
print(f"[ok] HCB frames: {len(pngs_HCB)}  MP4: {mp4_HCB}")

# Side-by-side comparison MP4 (same frame index; assumes equal counts)
print("[make] side-by-side HC|HCB")
SBS_DIR = os.path.join(OUT_ROOT, "side_by_side"); ensure_dir(SBS_DIR)
sbs_mp4 = os.path.join(SBS_DIR, "HC_vs_HCB.mp4")
pairs = zip(pngs_HC, pngs_HCB)
with iio.get_writer(sbs_mp4, fps=12, codec="libx264") as w:
    for a,b in pairs:
        A = iio.imread(a); B = iio.imread(b)
        # equalize heights, then hstack
        h = max(A.shape[0], B.shape[0])
        wA, wB = A.shape[1], B.shape[1]
        padA = np.zeros((h, wA, 3), dtype=np.uint8); padA[:A.shape[0], :wA] = A
        padB = np.zeros((h, wB, 3), dtype=np.uint8); padB[:B.shape[0], :wB] = B
        sbs = np.hstack([padA, padB])
        sbs = pad_to_macroblocks(sbs, 16)
        w.append_data(sbs)
print(f"[ok] Side-by-side: {sbs_mp4}")

print("\n=== DONE ===")
print("Output root:", OUT_ROOT)
print("HC MP4     :", mp4_HC)
print("HCB MP4    :", mp4_HCB)
print("SBS MP4    :", sbs_mp4)

[backend] CuPy (GPU)
[run] HC (Man2-locked)
[ok] HC frames: 601  MP4: man2locked_HC_vs_HCB/HC/HC.mp4
[run] HCB (Man2-locked + B)
[ok] HCB frames: 601  MP4: man2locked_HC_vs_HCB/HCB/HCB.mp4
[make] side-by-side HC|HCB
[ok] Side-by-side: man2locked_HC_vs_HCB/side_by_side/HC_vs_HCB.mp4

=== DONE ===
Output root: man2locked_HC_vs_HCB
HC MP4     : man2locked_HC_vs_HCB/HC/HC.mp4
HCB MP4    : man2locked_HC_vs_HCB/HCB/HCB.mp4
SBS MP4    : man2locked_HC_vs_HCB/side_by_side/HC_vs_HCB.mp4
